In [70]:
import requests
import config
from datetime import datetime, timedelta


In [ ]:



def get_response():

    end_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    start_date = end_date - timedelta(days=1)

    api_url = config.api_url
    print(end_date, start_date)
    q_params = {
        'api_url': config.api_url,
        'client': config.client,
        'client_key': config.client_key,
        'start': start_date,
        'end': end_date
    }
    response = requests.get(api_url, params=q_params)
    return response.json()



2026-01-03 00:00:00 2026-01-02 00:00:00
[{'lti_user_id': 'ac77a81548ac57d675c2569d72bc9593', 'passback_params': "{'oauth_consumer_key': '', 'lis_result_sourcedid': 'course-v1:SkillFactory+DSPRMGU+2023_FEB:lms.skillfactory.ru-a5b56554977d4c40b9334188f24afc33:ac77a81548ac57d675c2569d72bc9593', 'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPRMGU+2023_FEB/xblock/block-v1:SkillFactory+DSPRMGU+2023_FEB+type@lti+block@a5b56554977d4c40b9334188f24afc33/handler_noauth/grade_handler'}", 'is_correct': None, 'attempt_type': 'run', 'created_at': '2026-01-02 06:39:44.388133'}, {'lti_user_id': '298a085abb2a05f6ea3d6a157b023536', 'passback_params': "{'oauth_consumer_key': '', 'lis_result_sourcedid': 'course-v1:SkillFactory+DST-3.0+28FEB2021:lms.skillfactory.ru-c6b049cbab984edd9cb81462132d6c97:298a085abb2a05f6ea3d6a157b023536'}", 'is_correct': None, 'attempt_type': 'run', 'created_at': '2026-01-02 07:07:07.822685'}, {'lti_user_id': '298a085abb2a05f6ea3d6a157b02